In [7]:
import pandas as pd
import numpy as np

In [53]:
def one_hot(y_train):
    ceal=np.max(y_train)+1
    temp=np.zeros([ceal,y_train.shape[1]])
    for i in range(y_train.shape[1]):
        k=y_train[0][i]                    # Just another temp variable
        temp[k][i]=1
    return temp

In [76]:
def softmax(z):
    return(np.exp(z)/(np.sum(np.exp(z),axis=0))) # axis=0 sum column wise

In [77]:
def relu(z):
    return(np.maximum(z,0.01*z))

In [ ]:
def tanh(z):
    return((np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z)))

In [ ]:
def cost(A,y):
    cost=np.sum(y*np.log(A))
    return(cost)

In [61]:
def model(no_hd_l,hd_unit,input_size,output_size,initialization=None):
    if(len(hd_unit)!=no_hd_l):
        return
    hd_unit=[input_size]+hd_unit
    hd_unit=hd_unit+[output_size]
    parameters=dict()
    
    if(initialization is None):
        for i in range(1,no_hd_l+2):
            parameters['W'+str(i)]=np.random.randn(hd_unit[i],hd_unit[i-1])
            parameters['b'+str(i)]=np.zeros([hd_unit[i],1])
        
    if(initialization is 'He'):
        for i in range(1,no_hd_l+2):
            parameters['W'+str(i)]=np.random.randn(hd_unit[i],hd_unit[i-1])*np.sqrt(2/hd_unit[i-1])
            parameters['b'+str(i)]=np.zeros([hd_unit[i],1])
        
    if(initialization is 'Xavier'):
        for i in range(1,no_hd_l+2):
            parameters['W'+str(i)]=np.random.randn(hd_unit[i],hd_unit[i-1])*np.sqrt(1/hd_unit[i-1])
            parameters['b'+str(i)]=np.zeros([hd_unit[i],1])
    return(parameters)

In [74]:
def feed_forward(parameters,X_train,no_hd_l,activation):
    cache1=dict()
    if(activation is 'relu'):
        cache1['Z1']=np.matmul(parameters['W1'],X_train)+parameters['b1']
        cache1['A1']=relu(cache1['Z1'])
        for i in range(2,no_hd_l+1):
            cache['Z'+str(i)]=np.matmul(parameters['W'+str(i)],cache['A'+str(i-1)])+parameters['b'+str(i)]
            cache['A'+str(i)]=relu(cache['Z'+str(i)])
    
    if(activation is 'sigmoid'):
        cache1['Z1']=np.matmul(parameters['W1'],X_train)+parameters['b1']
        cache1['A1']=sigmoid(cache1['Z1'])
        for i in range(2,no_hd_l+1):
            cache['Z'+str(i)]=np.matmul(parameters['W'+str(i)],cache['A'+str(i-1)])+parameters['b'+str(i)]
            cache['A'+str(i)]=sigmoid(cache['Z'+str(i)])
    
    if(activation is 'tanh'):
        cache1['Z1']=np.matmul(parameters['W1'],X_train)+parameters['b1']
        cache1['A1']=tanh(cache1['Z1'])
        for i in range(2,no_hd_l+1):
            cache['Z'+str(i)]=np.matmul(parameters['W'+str(i)],cache['A'+str(i-1)])+parameters['b'+str(i)]
            cache['A'+str(i)]=tanh(cache['Z'+str(i)])
        
    cache['Z'+str(no_hd_l+1)]=np.matmul(parameters['W'+str(no_hd_l+1)],cache['A'+str(no_hd_l+1-1)])+parameters['b'+str(no_hd_l+1)]
    cache['A'+str(no_hd_l+1)]=softmax(cache['Z'+str(no_hd_l+1)])
    return(cache['A'+str(no_hd_l+1)])

In [ ]:
def backward(parameters,no_hd_l):
    

In [8]:
df=pd.read_csv("MNIST dataset/train.csv")

In [44]:
X_train=np.array(df.iloc[:,1:]).T
y_train=np.array(df.iloc[:,0])
y_train=y_train.reshape(1,y_train.shape[0])
y_train=one_hot(y_train) #one hot
del df

In [71]:
parameters=model(no_hd_l=2,hd_unit=[50,25],input_size=X_train.shape[0],output_size=10,initialization='Xavier')

In [72]:
print(parameters['W1'].shape,parameters['W2'].shape,parameters['W3'].shape)
print(parameters['b1'].shape,parameters['b2'].shape,parameters['b3'].shape)

(50, 784) (25, 50) (10, 25)
(50, 1) (25, 1) (10, 1)
